# 一致预期 profit 表

In [1]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [88]:
import json
import pymysql
import pymssql
# MySQL
# bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
# dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}
# datayesdb={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"database":"datayesdb","user":"app_gaea_ro","password":"EQw6WquhnCKPp8Li","charset":"utf8"}

# researchrpt={"host":"db-researchrpt-ro.wmcloud.com","port":3306,"user":"app_gaea_ro","passwd":"UDVVkvULzakP2048","db":"research_rpt","charset":"utf8"}
# dev
# researchrpt={"host":"10.24.21.34","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}
# prd
researchrpt={"host":"db-researchrpt.wmcloud.com","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}

# SQLserver
# datayes={"server":"sh-dm-db05.datayes.com","port":1433,"database":"dyedb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}
datayesdb={"server":"10.21.139.72","port":1433,"database":"datayesdb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}



# bigdata_conn = MysqlConn(json.dumps(bigdata)).connect()
# dateyesdbp_conn = MysqlConn(json.dumps(dateyesdbp)).connect()
# datayesdb_conn = MysqlConn(json.dumps(datayesdb)).connect()
researchrpt_conn = MysqlConn(json.dumps(researchrpt)).connect()

datayesdb_conn = MssqlConn(json.dumps(datayesdb)).connect()
# datayes_conn = MssqlConn(json.dumps(datayes)).connect()


In [58]:
import pandas as pd
from datetime import datetime, timedelta

# R04

In [4]:
sql="""
select id, code, code_name, title, organ_id, author, create_date from der_report_research 
where code = 002493 and organ_id=2570 order by create_date desc"""

base_df = pd.read_sql(sql, researchrpt_conn)

c:\python27\lib\site-packages\pymysql\cursors.py:166: Warning: (1292, u"Truncated incorrect DOUBLE value: 'A00611'")
  result = self._query(query)


In [5]:
base_df

,id,code,code_name,title,organ_id,author,create_date
0,2107999,002493,荣盛石化,荣盛石化（002493）：费用拖累18年业绩，浙石化转向生产准备阶段,2570,"张樨樨,贾广博",2019-04-17
1,2095553,002493,荣盛石化,荣盛石化（002493）：下修2018年业绩指引，18Q4单季度预亏,2570,"张樨樨,贾广博",2019-02-11
2,2053543,002493,荣盛石化,荣盛石化（002493）：财务费用拖累二季度业绩，下半年盈利有望改善,2570,张樨樨,2018-08-19
3,2011606,002493,荣盛石化,"荣盛石化(002493)：1季报符合预期,浙石化全面建设",2570,张樨樨,2018-04-26
4,2006523,002493,荣盛石化,"荣盛石化(002493)：PTA渐成产业链优势环节,浙石化开启新一轮成长!",2570,张樨樨,2018-04-06
5,1180294,002493,荣盛石化,荣盛石化：Q3业绩符合预期，芳烃－PTA－涤纶一体化优势明显,2570,"李振永,李扬",2016-10-27


In [6]:
sql="""
select REPORT_SEARCH_ID, TIME_YEAR, FORECAST_PROFIT,FORECAST_INCOME_SHARE from der_report_subtable 
where report_search_id = 2107999"""

In [7]:

base_df = pd.read_sql(sql, researchrpt_conn)
base_df

,REPORT_SEARCH_ID,TIME_YEAR,FORECAST_PROFIT,FORECAST_INCOME_SHARE
0,2107999,2019,421618.0,0.67
1,2107999,2020,786056.0,1.25
2,2107999,2021,890949.0,1.42


In [8]:
# 每一篇研报对应多个年份的预测值


In [21]:
# base_df
filter_str = "CREATE_DATE" # 根据 num 列分开1，2 后， 需要从num=1的数据中筛选出日期为查询日期的数据
this_date_str = '2019-03-29'


primary_key=['CODE','ORGAN_ID','TIME_YEAR'] # 用于merge
last_key=['FORECAST_PROFIT', 'FORECAST_INCOME_SHARE', 'CREATE_DATE'] # 同一个机构上次对本证券做的预测报告的撰写日期 同一个机构上次对本证券做的预测报告的评级

In [22]:
sql = """
SELECT  id, fileguid, CODE, CODE_NAME, TITLE, AUTHOR, CREATE_DATE, ORGAN_ID, num
FROM (
	SELECT id, fileguid, CODE, CODE_NAME, TITLE, AUTHOR, CREATE_DATE, ORGAN_ID,
		   (@row_number:=
			   case when CODE = @pre_code and ORGAN_ID = @pre_organ_id
			then @row_number+1
			 else 1
			end
		   ) as num,
		(@pre_code := CODE),
		(@pre_organ_id := ORGAN_ID)
	FROM der_report_research,(select @row_number:=1,@pre_code:='',@pre_organ_id:='') a
	where CREATE_DATE <= '%s 00:00:00'
	order by CODE,ORGAN_ID,CREATE_DATE desc
) a
where a.num<=2
""" % (this_date_str)
base_df = pd.read_sql(sql, researchrpt_conn)


In [23]:
base_df['num'] = base_df['num'].astype(int)
base_df[filter_str]=base_df[filter_str].dt.strftime('%Y-%m-%d')

In [12]:
# sql

In [13]:
base_df.tail(10)
# base_df["WRITE_DATE"] = base_df["WRITE_DATE"].astype(str)
# base_df.info()
# base_df[base_df[filter_str]=='2018-11-30']
# base_df[base_df['WRITE_DATE']=='2018-11-30']

,id,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID,num
168342,2104914,3700546,A19028,科前生物,科前生物（A19028）：科创板医药生物系列之一,杜永宏,2019-04-08,123,1
168343,2110499,3717230,A19031,澜起科技,澜起科技（A19031）电子行业科创板系列报告之二：内存接口芯片设计公司,"宋江波,胡慧",2019-04-24,216,1
168344,2114075,3742186,A19036,晶丰明源,晶丰明源（A19036）：LED驱动芯片国内领先，深耕通用领域发力智能照明,"黄瑜,季清斌",2019-05-06,20,1
168345,2104827,3700953,A19042,恒安嘉新,恒安嘉新（A19042）：科创板信息安全系列之一,于芳,2019-04-09,123,1
168346,2111670,3722040,A19063,心脉医疗,心脉医疗（A19063）：主动脉介入行业领先企业，多产品份额领先,"熊超逸,闫天一",2019-04-29,15467,1
168347,2111772,3721992,A19074,佰仁医疗,佰仁医疗（A19074）：专注动物源性植介入领域，多产品国内领先,"熊超逸,闫天一",2019-04-29,15467,1
168348,2111671,3721981,A19074,佰仁医疗,佰仁医疗（A19074）：专注动物源性植介入领域，多产品国内领先,"熊超逸,闫天一",2019-04-29,15467,2
168349,2111680,3722013,A19090,硕世生物,硕世生物（A19090）：掌握多重荧光定量PCR技术，体外诊断领先企业,"熊超逸,闫天一",2019-04-29,15467,1
168350,2115322,3761865,A19098,建龙微纳,建龙微纳（A19098）：科创板新材料企业解读：建龙微纳,刘曦,2019-05-14,77,1
168351,2039869,2115224,TEDU,达内科技,"达内科技(TEDU)：变革与突破,回溯达内15载王者进阶之路",黄莞,2018-01-23,10520,1


In [190]:
# # 根据fileguid取预测值
# # base_df["id"] = base_df["id"].astype(str)
# sql="""
# select REPORT_SEARCH_ID, TIME_YEAR, FORECAST_PROFIT,FORECAST_INCOME_SHARE from der_report_subtable 
# where report_search_id in (%s)""" % (','.join(list(base_df["id"].astype(str).values)))

In [191]:
# df_forecast = pd.read_sql(sql, researchrpt_conn)

In [264]:
# df_forecast.head(1)

In [194]:
# df_forecast = df_forecast.rename(columns={"REPORT_SEARCH_ID":"id"})

In [195]:
# df_final = pd.merge(base_df, df_forecast, on=['id'])

In [232]:
# df_final.head(1)
# # df_final[df_final.num==2]
# # FORECAST_PROFIT FORECAST_INCOME_SHARE

In [202]:
# df_final=df_final.drop("id",1)

In [24]:
# 用num的值区分次序
last_df = base_df[base_df["num"]==2]
last_df = last_df.drop("num", 1)

In [25]:
last_df.head(5)

,id,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID
24,1051783,529386E8-8E76-4E6D-8489-EFC47D611386,000001,平安银行,平安银行：负债结构优化，投行业务成为中收的核心驱动力，维持买入评级,"陆怡雯,高燕芸",2014-08-14,1
26,2083302,2260913,000001,平安银行,平安银行（000001）：包袱有望出清，从此轻装前行,"邱冠华,梁凤洁,袁梓芳",2018-11-17,2
28,2099592,3669772,000001,平安银行,平安银行（000001）：“量变”到“质变”，逐步确认转型进程,"张帅帅,王瑶平,严佳卉",2019-03-03,3
30,1079468,51D635DA-C1C5-4515-8B16-B31015B37C6D,000001,平安银行,平安银行2014年年报点评：零售业务平衡在收益与风险之间,王剑,2015-03-13,5
32,1207602,ADF6C7A3-A554-476A-962A-2D51F0243FB2,000001,平安银行,平安银行：窥一斑而知全豹，银行利润增速快于营业收入2017年的大趋势,"马鲲鹏,何悦",2017-04-21,7


In [26]:
last_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66825 entries, 24 to 167083
Data columns (total 8 columns):
id             66825 non-null int64
fileguid       66825 non-null object
CODE           66825 non-null object
CODE_NAME      66825 non-null object
TITLE          66825 non-null object
AUTHOR         66824 non-null object
CREATE_DATE    66825 non-null object
ORGAN_ID       66825 non-null int64
dtypes: int64(2), object(6)
memory usage: 4.6+ MB


In [27]:

this_df = base_df[(base_df["num"]==1) & (base_df[filter_str]==this_date_str)]
this_df = this_df.drop("num", 1)


In [28]:
this_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 1377 to 166782
Data columns (total 8 columns):
id             263 non-null int64
fileguid       263 non-null object
CODE           263 non-null object
CODE_NAME      263 non-null object
TITLE          263 non-null object
AUTHOR         263 non-null object
CREATE_DATE    263 non-null object
ORGAN_ID       263 non-null int64
dtypes: int64(2), object(6)
memory usage: 18.5+ KB


In [33]:
this_df.head()

,id,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID
1377,2103332,3692483,000039,中集集团,中集集团（000039）：营收创历史新高，海工业务将成未来业绩关键,刘兰程,2019-03-29,19
2209,2103064,3690913,000063,中兴通讯,中兴通讯（000063）公司年报点评：5G龙头归来，网络设备业务恢复、预计2019年Q1业绩高增长,"朱劲松,余伟民",2019-03-29,9
3939,2103164,3691730,000338,潍柴动力,潍柴动力（000338）2018年年报点评：业绩稳增长，加快新动能研发,卢嘉鹏,2019-03-29,8136
3989,2105694,3691196,000400,许继电气,许继电气（000400）2018年年报点评：业绩符合预期，新一轮高速增长将至,弓永峰,2019-03-29,13
3995,2103322,3692452,000400,许继电气,许继电气（000400）：核心业务即将重回高增长,王革,2019-03-29,20


In [29]:
# 用this_df 对last_df中的数据做筛选， 筛选完了再去查：select REPORT_SEARCH_ID, TIME_YEAR, FORECAST_PROFIT,FORECAST_INCOME_SHARE from der_report_subtable 
# 不然数据量太大
last_df_slim = pd.merge(last_df, this_df[['CODE', 'ORGAN_ID']], on=['CODE', 'ORGAN_ID'], how='right')
# last_df['id']=last_df['id'].astype(int)

In [30]:
last_df_slim = last_df_slim.dropna(subset=['id'])
last_df_slim['id']=last_df_slim['id'].astype(int)
# last_df.info()

In [31]:
last_df_slim.head(1)

,id,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID
0,2080358,2255968,000039,中集集团,中集集团（000039）：营收稳健增长，毛利率水平继续改善,"刘兰程,范想想",2018-10-30,19


In [32]:
last_df_slim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 244
Data columns (total 8 columns):
id             245 non-null int32
fileguid       245 non-null object
CODE           245 non-null object
CODE_NAME      245 non-null object
TITLE          245 non-null object
AUTHOR         245 non-null object
CREATE_DATE    245 non-null object
ORGAN_ID       245 non-null int64
dtypes: int32(1), int64(1), object(6)
memory usage: 16.3+ KB


In [303]:
# this_df.info()

In [304]:
if len(this_df) == 0:
    # 说明没有目标数据， 可以返回
    pass


In [34]:
ids=list(last_df_slim["id"].astype(str).values)
# ids

In [35]:
# 根据fileguid取预测值
# base_df["id"] = base_df["id"].astype(str)
sql="""
select REPORT_SEARCH_ID, TIME_YEAR, FORECAST_PROFIT,FORECAST_INCOME_SHARE from der_report_subtable 
where report_search_id in (%s)""" % (','.join(ids))

df_forecast = pd.read_sql(sql, researchrpt_conn)
df_forecast.head(1)

,REPORT_SEARCH_ID,TIME_YEAR,FORECAST_PROFIT,FORECAST_INCOME_SHARE
0,880740,2011,NaN,0.45


In [36]:
df_forecast = df_forecast.rename(columns={"REPORT_SEARCH_ID":"id"})


In [37]:
last_df_slim = pd.merge(last_df_slim, df_forecast, on=['id'])
last_df_fore=last_df_slim.drop("id",1)

In [43]:
last_df_fore.head()

,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID,TIME_YEAR,FORECAST_PROFIT,FORECAST_INCOME_SHARE
0,2255968,000039,中集集团,中集集团（000039）：营收稳健增长，毛利率水平继续改善,"刘兰程,范想想",2018-10-30,19,2018,NaN,0.91
1,2255968,000039,中集集团,中集集团（000039）：营收稳健增长，毛利率水平继续改善,"刘兰程,范想想",2018-10-30,19,2019,NaN,1.10
2,2255968,000039,中集集团,中集集团（000039）：营收稳健增长，毛利率水平继续改善,"刘兰程,范想想",2018-10-30,19,2020,NaN,1.29
3,2253874,000063,中兴通讯,中兴通讯（000063）：公司季报点评：研发提速加码5G，经营数据向好,"朱劲松,余伟民",2018-10-28,9,2018,-620709.0,-1.48
4,2253874,000063,中兴通讯,中兴通讯（000063）：公司季报点评：研发提速加码5G，经营数据向好,"朱劲松,余伟民",2018-10-28,9,2019,491837.0,1.17


In [49]:
last_df_fore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 0 to 707
Data columns (total 10 columns):
fileguid                 708 non-null object
CODE                     708 non-null object
CODE_NAME                708 non-null object
TITLE                    708 non-null object
AUTHOR                   708 non-null object
CREATE_DATE              708 non-null object
ORGAN_ID                 708 non-null int64
TIME_YEAR                708 non-null int64
FORECAST_PROFIT          692 non-null float64
FORECAST_INCOME_SHARE    702 non-null float64
dtypes: float64(2), int64(2), object(6)
memory usage: 60.8+ KB


In [38]:
ids= list(this_df["id"].astype(str).values)

In [39]:
# 根据fileguid取预测值
# base_df["id"] = base_df["id"].astype(str)
sql="""
select REPORT_SEARCH_ID, TIME_YEAR, FORECAST_PROFIT,FORECAST_INCOME_SHARE from der_report_subtable 
where report_search_id in (%s)""" % (','.join(ids))

df_forecast = pd.read_sql(sql, researchrpt_conn)
df_forecast.head(1)

,REPORT_SEARCH_ID,TIME_YEAR,FORECAST_PROFIT,FORECAST_INCOME_SHARE
0,2102997,2019,222700.0,1.2


In [40]:
df_forecast = df_forecast.rename(columns={"REPORT_SEARCH_ID":"id"})


In [41]:
this_df_fore = pd.merge(this_df, df_forecast, on=['id'])
this_df_fore=this_df_fore.drop("id",1)

In [44]:
this_df_fore.head(3)

,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,ORGAN_ID,TIME_YEAR,FORECAST_PROFIT,FORECAST_INCOME_SHARE
0,3692483,000039,中集集团,中集集团（000039）：营收创历史新高，海工业务将成未来业绩关键,刘兰程,2019-03-29,19,2019,NaN,1.25
1,3692483,000039,中集集团,中集集团（000039）：营收创历史新高，海工业务将成未来业绩关键,刘兰程,2019-03-29,19,2020,NaN,1.41
2,3692483,000039,中集集团,中集集团（000039）：营收创历史新高，海工业务将成未来业绩关键,刘兰程,2019-03-29,19,2021,NaN,1.57


In [50]:
this_df_fore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 0 to 754
Data columns (total 10 columns):
fileguid                 755 non-null object
CODE                     755 non-null object
CODE_NAME                755 non-null object
TITLE                    755 non-null object
AUTHOR                   755 non-null object
CREATE_DATE              755 non-null object
ORGAN_ID                 755 non-null int64
TIME_YEAR                755 non-null int64
FORECAST_PROFIT          744 non-null float64
FORECAST_INCOME_SHARE    749 non-null float64
dtypes: float64(2), int64(2), object(6)
memory usage: 64.9+ KB


In [51]:
key = last_key + primary_key
key

['FORECAST_PROFIT',
 'FORECAST_INCOME_SHARE',
 'CREATE_DATE',
 'CODE',
 'ORGAN_ID',
 'TIME_YEAR']

In [46]:
merge_df = pd.merge(this_df_fore, last_df_fore[key], how='left', on=primary_key)

In [47]:
# merge 获取上一次最近的数据时， 若上一次数据为空， 删除它们
# merge_df = merge_df.dropna(subset=['WRITE_DATE_y'])
primary_key

['CODE', 'ORGAN_ID', 'TIME_YEAR']

In [54]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457 entries, 0 to 753
Data columns (total 13 columns):
fileguid                   457 non-null object
CODE                       457 non-null object
CODE_NAME                  457 non-null object
TITLE                      457 non-null object
AUTHOR                     457 non-null object
CREATE_DATE_x              457 non-null object
ORGAN_ID                   457 non-null int64
TIME_YEAR                  457 non-null int64
FORECAST_PROFIT_x          449 non-null float64
FORECAST_INCOME_SHARE_x    456 non-null float64
FORECAST_PROFIT_y          448 non-null float64
FORECAST_INCOME_SHARE_y    454 non-null float64
CREATE_DATE_y              457 non-null object
dtypes: float64(4), int64(2), object(7)
memory usage: 50.0+ KB


In [337]:
merge_df.head(1)

,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE_x,ORGAN_ID,TIME_YEAR,FORECAST_PROFIT_x,FORECAST_INCOME_SHARE_x,FORECAST_PROFIT_y,FORECAST_INCOME_SHARE_y,CREATE_DATE_y
0,2145721,None,None,渤海证券研究所晨会,渤海证券研究所,2018-03-29,8,2018,159100.0,1.47,NaN,NaN,NaN


In [53]:
#merge_df 删除last date为空的记录
merge_df = merge_df.dropna(subset=['CREATE_DATE_y'])

In [338]:
# merge_df.to_csv("write_date_2019-03-21.csv", index=False, encoding='utf-8')

In [109]:
# cmb_report_research.ORGAN_ID=report_org_map.TGT_ORG_ID
sql = """SELECT distinct TGT_ORG_ID,TGT_ORG_NAME, TEXT1 FROM report_org_map"""
refer_df = pd.read_sql(sql, researchrpt_conn)

In [110]:
refer_df = refer_df.rename(columns = {"TGT_ORG_ID":"ORGAN_ID"})

In [111]:
refer_df.head()

,ORGAN_ID,TGT_ORG_NAME,TEXT1
0,118,中泰证券,14748
1,153,东吴证券,26959
2,13,中信证券,625
3,2570,天风证券,15750
4,122,国海证券,321


In [112]:
refer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
ORGAN_ID        343 non-null int64
TGT_ORG_NAME    343 non-null object
TEXT1           343 non-null object
dtypes: int64(1), object(2)
memory usage: 8.1+ KB


In [117]:
refer_df[refer_df['ORGAN_ID']==41984]

,ORGAN_ID,TGT_ORG_NAME,TEXT1
22,41984,国金证券(香港),179323
220,41984,国金证券（香港）,179323


In [116]:
df = refer_df.groupby(['ORGAN_ID']).size().to_frame(name='counts').reset_index()
df[df['counts']>1]

,ORGAN_ID,counts
242,41984,2


In [113]:
# refer_df.info()
# len(refer_df["TEXT1"].unique())
len(refer_df["ORGAN_ID"].unique())

342

In [94]:
# 去重， 存在不同org name 对应同
refer_df.drop_duplicates(['ORGAN_ID'],inplace=True)

In [98]:
# final_df.head()
final_df = pd.merge(merge_df, refer_df, how='left', on='ORGAN_ID') 
final_df = final_df.drop('ORGAN_ID', axis=1)
# final_df = pd.merge(merge_df, refer_df, how='left', on=['SEC_CODE', 'WRITE_DATE_x']) 

In [343]:
final_df.head(1)

,fileguid,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE_x,TIME_YEAR,FORECAST_PROFIT_x,FORECAST_INCOME_SHARE_x,FORECAST_PROFIT_y,FORECAST_INCOME_SHARE_y,CREATE_DATE_y,SRC_ORG_NAME,TEXT1
0,2145721,None,None,渤海证券研究所晨会,渤海证券研究所,2018-03-29,2018,159100.0,1.47,NaN,NaN,NaN,渤海证券,26946


In [221]:
# # rename columns 
# LAST_WRITE_DATE
# THIS_PROFIT
# LAST_PROFIT
# PROFIT_ADJUST_PER
# PROFIT_ADJUST_MARK
# THIS_EPS
# LAST_EPS
# EPS_ADJUST_MARK

final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","AUTHOR","THIS_WRITE_DATE","FORE_YEAR","THIS_PROFIT","THIS_EPS","LAST_PROFIT","LAST_EPS","LAST_WRITE_DATE","ORG_NAME","ORG_ID"]

In [140]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R04"

In [145]:
import numpy as np
def cal_profit_per(row):
    this = row[0]
    last = row[1]
    if this is np.nan or last is np.nan or last == 0 :
        return np.nan
    else:
        return this/last

In [146]:
final_df["PROFIT_ADJUST_PER"]=final_df[['THIS_PROFIT', 'LAST_PROFIT']].apply(cal_profit_per, axis=1)

In [142]:
# 取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知

final_df["PROFIT_ADJUST_MARK"] = final_df[["THIS_PROFIT", "LAST_PROFIT"]].apply(compare, axis=1)

NameError: name 'compare' is not defined

In [142]:
# 取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知

final_df["EPS_ADJUST_MARK"] = final_df[["THIS_EPS", "LAST_EPS"]].apply(compare, axis=1)

NameError: name 'compare' is not defined

In [69]:
# final_df["THIS_TAR_PRICE"] = final_df["THIS_TAR_PRICE"].fillna(0).astype(int)
# final_df["LAST_TAR_PRICE"] = final_df["LAST_TAR_PRICE"].fillna(0).astype(int)

In [72]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 227
Data columns (total 14 columns):
REPORT_ID          228 non-null int64
SEC_CODE           227 non-null object
SEC_NAME           228 non-null object
TITLE              228 non-null object
AUTHOR             228 non-null object
THIS_WRITE_DATE    228 non-null object
THIS_TAR_PRICE     81 non-null object
ORG_ID             228 non-null int64
ORG_NAME           228 non-null object
LAST_WRITE_DATE    110 non-null object
LAST_TAR_PRICE     36 non-null object
CURRENT_PRICE      178 non-null float64
REPORT_TYPE        228 non-null object
TAR_PRICE_MARK     228 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 26.7+ KB
